In [ ]:
import warnings
import math

import astropy.units as u
import astropy.coordinates as coord
from astropy.constants import G as _G
import emcee
import numpy as np
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline

from scipy.misc import derivative
from scipy.integrate import quad
from scipy.interpolate import interp1d
from scipy.optimize import root

In [ ]:
G = _G.decompose(galactic).value

In [ ]:
# Hernquist background potential
m_h = 1E11 # Msun
a = 20. # kpc
_hernquist = gp.HernquistPotential(m=m_h, c=a, units=galactic)

# Plummer density profile
m_p = 1E10 # Msun
b = 5 # kpc
_plummer = gp.PlummerPotential(m=m_p, b=b, units=galactic)

In [ ]:
def hernquist_potential(r):
    return -G*m_h / (r + a)

def hernquist_r2(phi):
    return (G*m_h/phi)**2 + a**2 + 2*G*m_h*a/phi
hernquist_r2(-0.012)

In [ ]:
def plummer_density(phi):
    r2 = hernquist_r2(phi)
    return 3/(4*np.pi*b**3) * (1+r2/b**2)**-2.5
plummer_density(-0.012)

In [ ]:
phi = -0.012
derivative(plummer_density, phi, dx=1E-3*phi)

In [ ]:
def integrand(phi, H):
    dp_dphi = derivative(plummer_density, phi, dx=1E-3*phi)
    return dp_dphi / np.sqrt(phi - H)

In [ ]:
some_E = -0.02
phi - some_E

In [ ]:
integrand(phi, some_E)

In [ ]:
quad(integrand, some_E, 0, args=(some_E,))

In [ ]:
f = lambda H: quad(integrand, H, 0, args=(H,))[0]
f(some_E)

In [ ]:
some_E = -0.01
derivative(lambda H: quad(integrand, H, 0, args=(H,))[0], some_E, dx=np.abs(1E-4*some_E))

In [ ]:
n_df = 2048
curlyE = np.linspace(1E-2,1,n_df)
energy_grid = -curlyE * G*m_h/a

df = np.zeros(n_df)
for i,energy in enumerate(energy_grid):
    df[i] = derivative(lambda H: quad(integrand, H, 0, args=(H,))[0], energy, dx=np.abs(1E-4*energy))
    
log_df = np.log(df)

In [ ]:
plt.semilogy(curlyE, df)

In [ ]:
log_df_interp = interp1d(energy_grid[np.isfinite(log_df)], log_df[np.isfinite(log_df)], 
                         fill_value="extrapolate")

In [ ]:
log_df_interp(-0.2*G*m_h/a)

## First, sample radii in a Plummer profile

In [ ]:
def sample_radii(size=1, r_min=0.*u.kpc, r_max=np.inf*u.kpc):
    r_min = r_min.to(u.kpc).value
    r_max = r_max.to(u.kpc).value
    
    # Plummer-specific
    Menc = lambda r: _plummer.c_instance.mass_enclosed(np.array([[r,0,0]]), G=_plummer.G)[0]
    Mtot = m_p
    
    def root_func(r, m):
        return (m - Menc(float(r))/Mtot)
    
    if r_min == 0.:
        m_min = 0.
    else:
        m_min = Menc(r_min)/Mtot
        
    if r_max == np.inf:
        m_max = 1.
    else:
        m_max = Menc(r_max)/Mtot
    
    m = np.random.uniform(m_min, m_max, size=size)
    return np.array([root(root_func, 1., args=(m[i],)).x[0] for i in range(size)])

In [ ]:
r = sample_radii(4096)
xyz = np.zeros((3,r.size))
xyz[0] = r

### Make sure the sampled radii follow the correct profile

In [ ]:
bins = np.logspace(-1, 3, 32)
H,_ = np.histogram(r, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.

q = np.zeros((3,len(bin_cen)))
q[0] = bin_cen
plt.plot(bin_cen, _plummer.density(q) / m_p, marker=None, lw=2., ls='--')

plt.loglog(bin_cen, H/V/r.size, marker=None)

plt.xlabel('$r$')
plt.ylabel('$n(r)$')

## now sample velocities from the DF...

In [ ]:
def ln_vel_dist(p, r):
    v = p[0]
    if v <= 0.:
        return -np.inf
    
    E = 0.5*v**2 + hernquist_potential(r)
    log_f = log_df_interp(E) + 2*np.log(v)

    return log_f

In [ ]:
nwalkers = 32
v = np.zeros_like(r)

with warnings.catch_warnings():
    warnings.filterwarnings('error')
    
    for i in range(len(r)):
        p0 = np.abs(np.random.normal(1E-3, 1E-4, (nwalkers,1))) # velocity!!
        sampler = emcee.EnsembleSampler(nwalkers=nwalkers, dim=1, lnpostfn=ln_vel_dist, args=(r[i],))
        
        try:
            _ = sampler.run_mcmc(p0, 128)
        except Warning:
            print("Failed!", i)
            break
        v[i] = sampler.chain[0,-1,0]

In [ ]:
for link in sampler.chain:
    plt.plot(link[:,0], drawstyle='steps', alpha=0.4, marker=None)

In [ ]:
plt.hist(v, bins=np.linspace(0, 0.3, 20))
plt.xlabel("$v$")

In [ ]:
def r_v_to_3d(r, v):
    phi = np.random.uniform(0, 2*np.pi, size=r.size)
    theta = np.arccos(2*np.random.uniform(size=r.size) - 1)
    sph = coord.PhysicsSphericalRepresentation(phi=phi*u.radian, theta=theta*u.radian, r=r*u.one)
    xyz = sph.represent_as(coord.CartesianRepresentation).xyz
    
    phi_v = np.random.uniform(0, 2*np.pi, size=v.size)
    theta_v = np.arccos(2*np.random.uniform(size=v.size) - 1)
    v_sph = coord.PhysicsSphericalRepresentation(phi=phi_v*u.radian, theta=theta_v*u.radian, r=v*u.one)
    v_xyz = v_sph.represent_as(coord.CartesianRepresentation).xyz
    
    return xyz, v_xyz

In [ ]:
xyz, vxyz = r_v_to_3d(r, v)

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=xyz, vel=vxyz)

In [ ]:
t_cross = r / v
ecc = np.zeros_like(t_cross)
r_f = np.zeros_like(t_cross)

for i in range(len(t_cross)):
    w = _hernquist.integrate_orbit(w0[i], dt=t_cross[i]/100., n_steps=2000)
    ecc[i] = w.eccentricity()
    r_f[i] = np.sqrt(np.sum(w.pos[:,-1]**2)).value

In [ ]:
plt.hist(ecc[np.isfinite(ecc)])

## See what final radial distribution looks like

In [ ]:
bins = np.logspace(-1, 3, 32)
H,_ = np.histogram(r_f, bins=bins)

V = 4/3*np.pi*(bins[1:]**3 - bins[:-1]**3)
bin_cen = (bins[1:]+bins[:-1])/2.

q = np.zeros((3,len(bin_cen)))
q[0] = bin_cen
plt.plot(bin_cen, _plummer.density(q) / m_p, marker=None, lw=2., ls='--')

plt.loglog(bin_cen, H/V/r.size, marker=None)

plt.xlabel('$r$')
plt.ylabel('$n(r)$')